In [1]:
!pip uninstall fitz PyMuPDF --yes
!pip install PyMuPDF
#!pip install PyMuPDF==1.19.1  ## install a release later than 1.19.0
#!pip install PyMuPDF==1.18.19  ## don't install 1.19.0 because of Wrong x0/y0 values for bboxes in get_text("words"), see: https://github.com/pymupdf/PyMuPDF/issues/1328
# ## Releases of PyMuPDF as of Oct 18, 2021: 1.11.2, 1.12.5, 1.13.20, 1.14.19.post2, 1.14.20, 1.14.21, 1.16.0, 1.16.1, 1.16.2, 1.16.3, 1.16.4, 1.16.5, 1.16.6, 1.16.7, 1.16.8, 1.16.9, 1.16.10, 1.16.11, 1.16.12, 1.16.13, 1.16.14, 1.16.15, 1.16.16, 1.16.17, 1.16.18, 1.17.0, 1.17.1, 1.17.2, 1.17.3, 1.17.4, 1.17.5, 1.17.6, 1.17.7, 1.18.0, 1.18.1, 1.18.2, 1.18.3, 1.18.4, 1.18.5, 1.18.6, 1.18.7, 1.18.8, 1.18.9, 1.18.10, 1.18.11, 1.18.12, 1.18.13, 1.18.14, 1.18.15, 1.18.16, 1.18.17, 1.18.18, 1.18.19, 1.19.0


Found existing installation: PyMuPDF 1.19.6
Uninstalling PyMuPDF-1.19.6:
  Successfully uninstalled PyMuPDF-1.19.6
  Using cached PyMuPDF-1.19.6-cp38-cp38-macosx_10_9_x86_64.whl (7.6 MB)
You should consider upgrading via the '/usr/local/anaconda3/bin/python -m pip install --upgrade pip' command.


In [10]:
!open -a Preview book-1.pdf book-2.pdf book-3.pdf

In [19]:
config = [
   {"filename" : "NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 1.pdf",
    "name" : "book-1",
    "page_count" : 642,
    "sections" : {
      "preface" : [1, 78],
      "toponymique" : [49, 76],
      "abbrev" : [77, 78],
      "content" : [79, 607],
      "sample" : [79, 79+10],
      "index" : [617, 639]},
    },
   {"filename" : "NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 2.pdf",
    "name" : "book-2",
    "page_count" : 725,
    "sections" : {
      "preface" : [1, 79],
      "abbrev" : [6, 7],
      "content" : [8, 700],
      "sample" : [8, 8+10],
      "index" : [704, 725]},
    },
   {"filename" : "NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 3.pdf",
    "name" : "book-3",
    "page_count" : 588,
    "sections" : {
      "preface" : [1, 7],
      "abbrev" : [6, 7],
      "content" : [8, 554],
      "sample" : [8, 8+10],
      "index" : [556, 583],
      "familyidx" : [584, 585]},
    },
]


In [ ]:
import sys, fitz, re
from pprint import pprint

## Purpose: this cell is a pre-processing step that will create a symlink (e.g. book-1.pdf)
##          that points to the original file, and then will create new PDF documents
##          that are excerpts of subsets of the whole book
##          as named sections such as book-1-index.pdf or book-1-abbrev.pdf
##          The file book-X-sample.pdf contains the first 10 pages of the book-1-content.pdf
##          and the sample is used to speed up development/testing later on

## https://pymupdf.readthedocs.io/en/latest/document.html#Document.delete_pages
##    In general, the performance of this method is dependent on the number of remaining pages
##    NOT on the number of deleted pages
## So, keeping fewer pages is faster, and deleting more pages at once is also faster 
def excerpt(infile, outfile, start, end):
  doc = fitz.open(infile)  # open document
  last_page = doc.page_count
  print(start, end, last_page, outfile)
  deleteme = []
  if start - 1 > 0:
    deleteme.extend(list(range(0, start - 1)))       
  if end < last_page:
    deleteme.extend(list(range(end, last_page)))
  # print(deleteme)
  doc.delete_pages(deleteme)
  doc.save(outfile, garbage=4, clean=True)

import os, os.path
for book in config:
   symlink = book["name"] + ".pdf"
   fname = book["filename"]
   if os.path.exists(book["filename"]) and not os.path.exists(symlink):
      print("Creating symlink: ", symlink, " ->", fname)
      os.symlink(book["filename"], symlink)
   for section in book["sections"]:
      #print(s)
      start, end = book["sections"][section];
      out_fname = book["name"] + "-" + section + ".pdf"
      excerpt(book["filename"], out_fname, start, end)

!open -a Preview book-[1,2,3]-*.pdf

## Runtime of this cell is ~11m38s

In [15]:
# !open -a Preview book-[1,2,3]-*.pdf


In [16]:
!open -a Preview book-[1,2,3]-index.pdf


In [44]:
# https://pymupdf.readthedocs.io/en/latest/faq/#how-to-analyze-font-characteristics

import fitz
import pprint, re

def flags_decomposer(flags):
    """Make font flags human readable."""
    l = []
    if flags & 2 ** 0:
        l.append("superscript")
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return ", ".join(l)

class Face:
    def __init__(self, font, size, color, flags=0):
        self.font = font
        self.size = size
        self.color = color
        self.flags = flags
        self.style = flags_decomposer(self.flags)
    def __str__(self):
        return f"Font: '{self.font}' ({self.style}), size {self.size:g}, color " + ("#%06x" % self.color)
print(Face("Times", 10.929, 0, 4))


Font: 'Times' (serifed, proportional), size 10.929, color #000000


In [45]:
import sys, fitz, re
from pprint import pprint
# fname = "book-1-sample.pdf"
fname = "book-1.pdf"
doc = fitz.open(fname)  # open document

from fitz.utils import getColor
#blue = getColor("aliceblue")

# txtpgoffset = 77
txtpgoffset = 0
# start_page = 1 + txtpgoffset
start_page = 1
#end_page = start_page + 11
#end_page = start_page + 150
#end_page = start_page + 530
end_page = doc.page_count
items = []
mode = "searching"
print(mode)
for i in range(start_page, end_page):
    page = doc[i]
    # read page text as a dictionary, suppressing extra spaces in CJK fonts
    flags = 0 | fitz.TEXT_INHIBIT_SPACES | fitz.TEXT_DEHYPHENATE
    blocks = page.get_text("dict", flags=flags)["blocks"]
    for x,b in enumerate(blocks):  # iterate through the text blocks
        br = fitz.Rect(b["bbox"])
        #annot = page.add_rect_annot(br)
        for y,l in enumerate(b["lines"]):  # iterate through the text lines
            lr = fitz.Rect(l["bbox"])
            for z,s in enumerate(l["spans"]):  # iterate through the text spans
                if z==0 and mode == "found":
                    mode = "spanning"
                #s["style"] = flags_decomposer(s["flags"])  # readable font flags
                face = Face(s["font"], s["size"], s["color"], s["flags"])
                t = s["text"]
                ## FIXME: [83, 'N  Cystopteris  filix-fragilis  (L.) Borb. —  Polypodiumfilix-fragile  L.,  Cyst,  fragilis']
                ## FIXMECLASS: OCR Error; a spurious mark in the text preceding the genus is interpreted as a letter 'N'
                if z==0 and mode != "found" and re.search(r"Times-Bold\b", str(face)) and s["size"] > 10.0:
                        r_rect = fitz.Rect(lr)
                        r_high = fitz.Rect(s["bbox"])
                        joined = " ".join(list(map(lambda x: x["text"].strip(), l["spans"])))  ## FIXMEDESC: ideally this simplistic code would work if not for OCR Errors
                        item = joined
                        # found = re.match(r"(?P<genus>([A-Z]\w+) (?P<species>\w+)", joined)
                        # d = found.groupdict()
                        matches = re.match(r"([A-Z]\w+) (\w+)", joined)
                        warning = "Possible OCR Error"
                        if matches and len(matches.groups()) == 2:
                            found = matches.groups()
                            warning = "Normal"
                            mode = "found"
                            # item = f"""${d['genus']} ${d['species']}"""
                            item = " ".join(found)
                        else:
                            warning = "Possible OCR Error"

                        print([i, x, y, z, item, warning])
                        annot_high = page.add_highlight_annot(r_high)
                        items.append([t, r_rect, r_high, i, z,s, y,l, x,b, warning])
print(len(items))
#print(i, items[0][:3])


searching
[34, 0, 0, 0, 'r', 'Possible OCR Error']
[59, 9, 0, 0, 'J', 'Possible OCR Error']
[77, 8, 5, 0, 'JL', 'Possible OCR Error']
[78, 8, 0, 0, 'Lycopodium cernuum', 'Normal']
[79, 1, 0, 0, 'Selaginella denticulata', 'Normal']
[79, 16, 0, 0, 'Isoetes hystrix', 'Normal']
[79, 24, 1, 0, 'Isoetes olympica', 'Normal']
[80, 13, 0, 0, 'Equisetum maximum', 'Normal']
[80, 18, 0, 0, 'Equisetum palustre', 'Normal']
[80, 25, 0, 0, 'Equisetum ramosissimum', 'Normal']
[81, 9, 0, 0, 'Ophioglossum vulgatum', 'Normal']
[81, 11, 3, 0, 'Ophioglossum lusitanicum', 'Normal']
[81, 21, 0, 0, 'Osmunda regalis', 'Normal']
[82, 9, 0, 0, 'Gymnogramma leptophylla', 'Normal']
[82, 17, 0, 0, 'Cheilanthes pteridioides', 'Normal']
[83, 28, 0, 0, 'Âdiantum capillus-veneris I (PL III, n. 4). — 2|. Rhizome rampant à écailles', 'Possible OCR Error']
[84, 10, 0, 0, 'Pteris longifolia', 'Normal']
[84, 16, 0, 0, 'Pteridium aquilmum', 'Normal']
[85, 1, 0, 0, 'Athyrium filix', 'Normal']
[85, 12, 0, 0, 'Dryopterîs aculeat

In [46]:
## This cell requires ~20 mins to calculate results from the first book

import math
doc = fitz.open(fname)  # open document
#subset = items[:6]
subset = items
boxes = []
for it, item in enumerate(subset):
    mode = "searching"
    if (it >= len(subset) - 1):
        break
    next_item = subset[it + 1]
    [t0, rr0, rh0, i0, z0,s0, y0,l0, x0,b0, warning0] = item
    [t, rr, rh, i, z,s, y,l, x,b, warning] = [t0, rr0, rh0, i0, z0,s0, y0,l0, x0,b0, warning0]
    [t1, rr1, rh1, i1, z1,s1, y1,l1, x1,b1, warning1] = next_item
    page = doc[i0]
    flags = 0 | fitz.TEXT_INHIBIT_SPACES | fitz.TEXT_DEHYPHENATE
    blocks = page.get_text("dict", flags=flags)["blocks"]
    blocks = blocks[x:]
    r_rect = rr0
    
    #print(t0)
    # if t0 == "Isoetes olympica":
    #print([int(it), len(subset), rr0, t0, "\n", t1])
    
    annot_rect = page.add_rect_annot(rr0)
    annot_rect.set_border(width=1, dashes=[1,2])
    annot_rect.update()

    
    for x,b in enumerate(blocks):  # iterate through the text blocks
        br = fitz.Rect(b["bbox"])
        #annot = page.add_rect_annot(br)
        for y,l in enumerate(b["lines"][y0:]):  # iterate through the text lines
            lr = fitz.Rect(l["bbox"])
            s = l["spans"][0]
            #for z,s in enumerate(l["spans"]):  # iterate through the text spans
            this_line = " ".join(list(map(lambda x: x["text"], l["spans"])))
            if mode != "found":
                if re.search("Aire g", this_line):
                    mode = s["font"] + " " + str(s["size"])
                    r_rect = r_rect.include_rect(lr)
                    #print(str((x, y, s["size"], s["font"])) + "FOUND: " + this_line)
                elif lr.intersects(rr1):                    
                    mode = "found"
                    #print(str((x, y, s["size"], s["font"])) + "FOUND: " + this_line)
                else:
                    if mode != "searching":
                        (font, size) = mode.split(" ")
                        isize = round(math.ceil(float(size)))
                        if s["size"] >= float(isize):
                            mode = "found"
                    if mode != "found":
                        r_rect = r_rect.include_rect(lr)
                        # if y >= y0:
                        #     r_rect = r_rect.include_rect(lr)
                    
    r_high = rh0
    annot_rect = page.add_rect_annot(r_rect)
    annot_high = page.add_highlight_annot(r_high)
    # boxes.append([t0, start_page + it, r_rect, item]) . ## FIXME: thinking it should be i0 not start_page + it
    boxes.append([t0, i0, r_rect, item])


# print(boxes[0])
print(len(boxes))

#doc.delete_pages(end_page, doc.page_count - 1)
#doc.delete_pages(0, start_page - 1)
marked_epithet_fname = "marked-pages-" + doc.name
doc.save(marked_epithet_fname, garbage=4, clean=True)
#!open -a Preview marked-pages-book-1-sample.pdf
!open -a Preview marked-pages-book-1.pdf



1064


In [50]:
import pprint
pp = pprint.PrettyPrinter(compact=True)

import math
doc = fitz.open(fname)  # open document
results = {}
for it, thing in enumerate(boxes):
    mode = "searching"
    #print(thing)
    [name, pageno, rrr, item] = thing
    page = doc[pageno]
    [t0, rr0, rh0, i0, z0,s0, y0,l0, x0,b0, warning0] = item
    flags = 0 | fitz.TEXT_INHIBIT_SPACES | fitz.TEXT_DEHYPHENATE
    blocks = page.get_text("dict", clip=rr0, flags=flags)["blocks"]    


    print(pageno - txtpgoffset, i0, t0)
    results[t0] = {}
    results[t0]["name"] = t0
    results[t0]["name_rects"] = [rr0]
    results[t0]["pdf_page"] = pageno
    results[t0]["book_page"] = pageno - txtpgoffset
    key = "Extra"
    mode = "searching"
    margins = [[(math.ceil(l["bbox"][0]), l["spans"][0]["text"].split(" ")[0]) for l in filter(lambda bl: bl.get("spans"), b["lines"])] for b in blocks]
    margins = [m for n in margins for m in n]
    targets = list(map(lambda n: n[0], filter(lambda m: m[1] in ["L.", "S.", "Aire"] , margins))) or [blocks[0]["lines"][0]["bbox"][0]]
    margins_max = max(targets or margins)
    margins_min = min(list(map(lambda n: n[0], margins)))

    for x,b in enumerate(blocks):  # iterate through the text blocks
        br = fitz.Rect(b["bbox"])
        for y,l in enumerate(b["lines"][y0:]):  # iterate through the text lines
            lr = fitz.Rect(l["bbox"])
            xloc = min(margins_max, math.ceil(lr.x0))
            this_line = re.sub(r"\s+", " ", " ".join(list(map(lambda x: x["text"].strip(), l["spans"])))).strip()
            # matches = re.match("^(?P<label>L\.|S\.|Aire g\w+\.|Not\w+\.|Fleurs|Fructif\.|Fructific|Floraison\w+)\s", this_line)
            #### FIXME: Phyllitis scolopendrium (L.) Newm. —Asplenium scolopendrium L., Scolopendrium
            #### FIXME: Juniper
            matches = re.match("^(?P<label>" + t0 + "|.*?)(\s|$)", this_line)
            if matches:
                d = matches.groupdict()
                if margins_max - xloc < 5:
                    key = d["label"]
                    if ' ' in key:
                        key = "Description"
                    elif key not in ["Description", "L.", "S.", "Aire"]:
                        key = "Extra"
                    elif key in ["Description", "L.", "S.", "Aire"]:
                        pass
                    else:
                        print("UNHANDLED: " + key)
                        key = "Extra"
                frag = results[t0].get(key) or ""
                results[t0][key] = " ".join([frag.strip(), this_line.strip()]).strip()
                key_rects = key + "_rects"
                if not results[t0].get(key_rects):
                    results[t0][key_rects] = []    
                results[t0][key_rects].append(lr)
                #print([x, y, xloc, mode, this_line])
            else:
                print(["ERROR:", x, y, xloc, mode, this_line])
    # pp.pprint(results[t0])
    print()
    #break
                    
    # r_high = rh0
    # annot_rect = page.add_rect_annot(r_rect)
    # annot_high = page.add_highlight_annot(r_high)
    # boxes.append([t0, r_rect, item])




34 34 r

59 59 J

77 77 JL

78 78 Lycopodium cernuum

79 79 Selaginella denticulata

79 79 Isoetes hystrix

79 79 Isoetes olympica

80 80 Equisetum maximum

80 80 Equisetum palustre

80 80 Equisetum ramosissimum

81 81 Ophioglossum vulgatum

81 81 Ophioglossum lusitanicum

81 81 Osmunda regalis

82 82 Gymnogramma leptophylla

82 82 Cheilanthes

83 83 Âdiantum capillus-veneris

84 84 Pteris longifolia

84 84 Pteridium aquilmum

85 85 Athyrium

85 85 Dryopterîs aculeata

85 85 Dryopteris

86 86 Blechnum

87 87 Ceterack officinarum

88 88 Asplenium bourgaei

88 88 Asplenium

88 88 Asplenium ruta-muraria

88 88 Asplenium

88 88 Asplenium

89 89 Phyllitis scolopendrium

89 89 Phyllitis hemionitis

90 90 Polypodium vulgare L.

90 90 Salvinia natans

92 92 Abies cilicica

92 92 Cedrus libani

93 93 Pinus pinea

93 93 Pina

93 93 Pinus halepensis

94 94 Cupressus sempervirens

95 95 Juniperus oxycedrus

96 96 Juniper

96 96 Juniperus

96 96 Arceuthos drupacea

97 97 Ephedra campylopoda

97 97 

In [51]:
from tqdm import tqdm
import math
# marked_fname = "marked-pages-book-1-sample.pdf"
marked_fname = "marked-pages-book-1.pdf"
#doc = fitz.open(fname)  # open document
doc = fitz.open(marked_fname)  # open document
print(len(results))

from fitz.utils import getColorList
cl = getColorList()
from fitz.utils import getColor

pink = getColor("lightpink")
green = getColor("aquamarine")
blue = getColor("lightskyblue")
gray = getColor("whitesmoke")
yellowish = getColor("antiquewhite")
fills= [pink, green, blue, gray, yellowish]

fc = {"L.":pink, "S.":blue, "Aire":green, "Description":gray, "Extra":yellowish}
for it, name in enumerate(tqdm(results.keys())):
    item = results[name]
    #print(item)
    #print(item["Description"])
    book_page = item["book_page"]
    pdf_page = item["pdf_page"]
    print([it, name, book_page, pdf_page])
    page = doc[book_page - 1]
    for k in filter(lambda x: not re.search(r"_page|_rect", x), item.keys()):
        rects = item[k + "_rects"]
        for r in rects:
            #print(k, r)
            
            if not page.rect.intersects(r):
                print(["ERROR: ", k, r])
            else:
                # #annot = page.add_highlight_annot(fitz.Rect(r))
                # annot_rect = page.add_rect_annot(fitz.Rect(r))
                # #annot_rect.set_colors(stroke=(0,1,0))
                # annot_rect.set_border(width=1, dashes=[1,2])
                # annot_rect.update()

                if k in ["L.", "S.", "Aire", "Description", "Extra"]:
                    print(["HIGHL: ", fc[k], k, r])
                    annot = page.add_highlight_annot(fitz.Rect(r))
                    annot.set_colors(stroke=fc[k])
                    annot.update()
                    #annot_rect = page.add_rect_annot(fitz.Rect(r))
                    #annot_rect.set_colors(stroke=green)
                    #annot_rect.set_border() #(width=1, dashes=[1,2])
                    #annot_rect.update()
                    #annot_rect.update(fill_color=red)


# doc.delete_pages(end_page, doc.page_count - 1)
# doc.delete_pages(0, start_page - 1)
doc_fname = "results-" + doc.name
doc.save(doc_fname, garbage=4, clean=True)
print(doc_fname)
# !open -a Preview results-marked-pages-book-1-sample.pdf
!open -a Preview results-marked-pages-book-1.pdf



  2%|▏         | 14/862 [00:00<00:06, 138.46it/s]

862
[0, 'r', 34, 34]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(19.200000762939453, -15.452678680419922, 30.185924530029297, 19.162628173828125)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(259.9200134277344, 48.01697540283203, 346.7517395019531, 62.690975189208984)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(515.729736328125, 48.01697540283203, 555.8638305664062, 62.690975189208984)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(76.08000183105469, 76.53803253173828, 556.5704956054688, 95.21402740478516)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(48.720001220703125, 91.89801788330078, 256.58343505859375, 110.57401275634766)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(76.08000183105469, 111.81800079345703, 556.623046875, 130

  6%|▌         | 51/862 [00:00<00:04, 173.60it/s]


['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(38.15999984741211, 545.8829956054688, 228.19778442382812, 557.8889770507812)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(38.15999984741211, 579.0029907226562, 282.1980285644531, 591.0089721679688)]
['HIGHL: ', (0.5294117647058824, 0.807843137254902, 0.9803921568627451), 'S.', Rect(38.15999984741211, 557.6430053710938, 405.2191467285156, 569.6489868164062)]
['HIGHL: ', (0.5294117647058824, 0.807843137254902, 0.9803921568627451), 'S.', Rect(20.8799991607666, 566.7229614257812, 99.93690490722656, 579.489013671875)]
['HIGHL: ', (0.4980392156862745, 1.0, 0.8313725490196079), 'Aire', Rect(37.68000030517578, 590.7630004882812, 313.6315002441406, 602.7689819335938)]
[30, 'Abies cilicica', 92, 92]
['HIGHL: ', (0.9607843137254902, 0.9607843137254902, 0.9607843137254902), 'Description', Rect(42.0, 182.57699584960938, 405.0628967285156, 197.9110107421875)]
['HIGHL: 

 11%|█         | 95/862 [00:00<00:04, 188.93it/s]

...


['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(14.399999618530273, 484.3630065917969, 399.1833190917969, 496.3689880371094)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(14.399999618530273, 493.7230224609375, 399.03985595703125, 505.72900390625)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(14.399999618530273, 503.0830078125, 399.1338195800781, 515.0889892578125)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(14.15999984741211, 512.4429931640625, 48.59714889526367, 524.448974609375)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(32.400001525878906, 526.60302734375, 399.1040344238281, 538.6090087890625)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(14.15999984741211, 536.2030029296875, 399.1568298339844, 548.208984375)]
['HIGHL: ', (0.9

 98%|█████████▊| 847/862 [00:06<00:00, 166.41it/s]


['HIGHL: ', (0.9607843137254902, 0.9607843137254902, 0.9607843137254902), 'Description', Rect(16.31999969482422, 260.9369812011719, 400.6160888671875, 275.6109924316406)]
['HIGHL: ', (0.9607843137254902, 0.9607843137254902, 0.9607843137254902), 'Description', Rect(15.84000015258789, 272.4570007324219, 401.01507568359375, 287.1310119628906)]
['HIGHL: ', (0.9607843137254902, 0.9607843137254902, 0.9607843137254902), 'Description', Rect(16.079999923706055, 283.97698974609375, 400.8644104003906, 298.6510009765625)]
['HIGHL: ', (0.9607843137254902, 0.9607843137254902, 0.9607843137254902), 'Description', Rect(15.84000015258789, 295.4969787597656, 400.8550720214844, 310.1709899902344)]
['HIGHL: ', (0.9607843137254902, 0.9607843137254902, 0.9607843137254902), 'Description', Rect(16.559999465942383, 307.0169982910156, 400.9403991699219, 321.6910095214844)]
['HIGHL: ', (0.9607843137254902, 0.9607843137254902, 0.9607843137254902), 'Description', Rect(16.079999923706055, 318.5369873046875, 249.892

100%|██████████| 862/862 [00:06<00:00, 129.32it/s]


['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(39.84000015258789, 560.4463500976562, 424.0772399902344, 572.0132446289062)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(39.599998474121094, 569.806396484375, 431.3534240722656, 581.373291015625)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(39.84000015258789, 579.1663818359375, 424.0456237792969, 590.7332763671875)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(38.400001525878906, 588.5263671875, 424.1494140625, 600.09326171875)]
['HIGHL: ', (0.9803921568627451, 0.9215686274509803, 0.8431372549019608), 'Extra', Rect(39.599998474121094, 597.8864135742188, 424.3891296386719, 609.4533081054688)]
[858, 'Dianthus judaicus', 603, 603]
['HIGHL: ', (0.9607843137254902, 0.9607843137254902, 0.9607843137254902), 'Description', Rect(55.91999816894531, 42.05701446533203, 418.85345458

In [53]:
import pandas as pd

df = pd.DataFrame.from_dict(results.values()).fillna('')
df = df[df.columns.drop(list(df.filter(regex='_rects')))]
df.to_csv("book-1.csv", index=False)
df

,name,pdf_page,book_page,Extra,Description,L.,Aire,S.
0,r,34,34,r INTRODUCTION XXXVII Occupant des aires très ...,,,,
1,J,59,59,J Lettre alternant dans la transcription de mo...,,,,
2,JL,77,77,"Nab, N Nahal Nik Np Ol P Pb Pr Reese Russ Sam ...",,,,
3,Lycopodium cernuum,78,78,Fructification de novembre à mars. Sur grès tr...,Lycopodium cernuum L. var. capillaceum Willd (...,"L. Mi. Entre Nahr es Safa et 'Aïn Qa'a, 1942 e...",Aire géogr. —- Régions tropicales et subtropic...,
4,Selaginella denticulata,79,79,Végétation active de novembre à avril. Fructif...,Selaginella denticulata (L.) Link — Lycopodium...,"L. Ct. et ML, Ce. Saïda (Bl), Beyrouth et envi...",Aire géogr. —• Tour de la Méditerranée. Madère...,"S. Non signalée. Présence presque certaine, Ct..."
...,...,...,...,...,...,...,...,...
857,Dianthus strictus,599,599,Floraison: mai-décembre. CC. tous terrains. Le...,Dianthus strictus Banks et Sol. (non Sibth. et...,,,
858,Dianthus judaicus,603,603,Var. auraniticus (Post.) n. comb. — Calice 3 c...,Dianthus judaicus Boiss. — D. pattens Sibth. e...,L. Sy. Baalbeck (Wall). St. Qamou'at Hermel (P...,"Aire géogr. — Turquie sud, Syrie, Liban, Pales...","S. A.L. Ouadi-el-Qarn (Sam, Mt, Pb), Zebdani, ..."
859,Dianthus pachypetalus,603,603,,Dianthus pachypetalus Stapf— D. floribundus Bo...,,,
860,Dianthus crinitus,604,604,Floraison: mai-juin.,"Dianthus crinitus Smith (PI. CLXXXV, n. 4). — ...",,"Aire géogr. — Turquie, Géorgie, Iran, Bélouchi...",S. St. Zélaf (Pb).
